# Udacity - Machine Learning Engineer Nanodegree
## Capstone Project
### Title: Development of a LSTM Network to Predict Students’ Answers on Exam Questions

### Implementation of DKT:
#### Part 1: Define constants

In [1]:
fn = "data/ASSISTments_skill_builder_data.csv" # Dataset path
verbose = 1 # Verbose = {0,1,2}
best_model_weights = "weights/bestmodel" # File to save the model.
log_dir = "logs" # Path to save the logs.
optimizer = "adam" # Optimizer to use
lstm_units = 100 # Number of LSTM units
batch_size = 32 # Batch size
epochs = 10 # Number of epochs to train
dropout_rate = 0.3 # Dropout rate
test_fraction = 0.2 # Portion of data to be used for testing
validation_fraction = 0.2 # Portion of training data to be used for validation

#### Part 2: Pre-processing

In [2]:
from deepkt import deepkt, data_util, metrics


dataset, length, nb_features, nb_skills = data_util.load_dataset(fn=fn,
                                                                 batch_size=batch_size,
                                                                 shuffle=True)

train_set, test_set, val_set = data_util.split_dataset(dataset=dataset,
                                                       total_size=length,
                                                       test_fraction=test_fraction,
                                                       val_fraction=validation_fraction)


set_sz = length * batch_size
test_set_sz = (set_sz * test_fraction)
val_set_sz = (set_sz - test_set_sz) * validation_fraction
train_set_sz = set_sz - test_set_sz - val_set_sz
print("============= Data Summary =============")
print("Total number of students: %d" % set_sz)
print("Training set size: %d" % train_set_sz)
print("Validation set size: %d" % val_set_sz)
print("Testing set size: %d" % test_set_sz)
print("Number of skills: %d" % nb_skills)
print("Number of features in the input: %d" % nb_features)
print("========================================")

============= Data Summary =============
Total number of students: 4160
Training set size: 2662
Validation set size: 665
Testing set size: 832
Number of skills: 123
Number of features in the input: 246


#### Part 3: Building the model

In [3]:
student_model = deepkt.DKTModel(
                        nb_features=nb_features,
                        nb_skills=nb_skills,
                        hidden_units=lstm_units,
                        dropout_rate=dropout_rate)

student_model.compile(
        optimizer=optimizer,
        metrics=[
            metrics.BinaryAccuracy(),
            metrics.AUC(),
            metrics.Precision(),
            metrics.Recall()
        ])

student_model.summary()

Model: "DKTModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, None, 246)]       0         
_________________________________________________________________
masking (Masking)            (None, None, 246)         0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         138800    
_________________________________________________________________
outputs (TimeDistributed)    (None, None, 123)         12423     
Total params: 151,223
Trainable params: 151,223
Non-trainable params: 0
_________________________________________________________________


#### Part 4: Train the Model

In [4]:
import tensorflow as tf
history = student_model.fit(dataset=train_set,
                            epochs=epochs,
                            verbose=verbose,
                            validation_data=val_set,
                            callbacks=[ 
                                tf.keras.callbacks.CSVLogger(f"{log_dir}/train.log"),
                                tf.keras.callbacks.ModelCheckpoint(best_model_weights,
                                                                   save_best_only=True,
                                                                   save_weights_only=True),
                                tf.keras.callbacks.TensorBoard(log_dir=log_dir)
        ])

Epoch 1/10
83/83 [==============================] - 12s 146ms/step - loss: 0.0657 - binary_accuracy: 0.7288 - auc: 0.7076 - precision: 0.7478 - recall: 0.9222 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
83/83 [==============================] - 9s 108ms/step - loss: 0.0557 - binary_accuracy: 0.7720 - auc: 0.8120 - precision: 0.7765 - recall: 0.9440 - val_loss: 0.0666 - val_binary_accuracy: 0.7878 - val_auc: 0.8344 - val_precision: 0.7908 - val_recall: 0.9487
Epoch 3/10
83/83 [==============================] - 9s 108ms/step - loss: 0.0612 - binary_accuracy: 0.7788 - auc: 0.8233 - precision: 0.7862 - recall: 0.9348 - val_loss: 0.0607 - val_binary_accuracy: 0.7715 - val_auc: 0.8289 - val_precision: 0.7733 - val_recall: 0.9310
Epoch 4/10
83/83 [==============================] - 9s 103ms/step - loss: 0.0596 - binary_accuracy: 0.7877 - auc: 0.8315 - precision: 0.7991 - recall: 0.9319 - val_loss:

#### Part 5: Load the Model with the Best Validation Loss

In [5]:
student_model.load_weights(best_model_weights)

#### Part 6: Test the Model

In [6]:
result = student_model.evaluate(test_set, verbose=verbose)

     26/Unknown - 2s 67ms/step - loss: 0.0529 - binary_accuracy: 0.8071 - auc: 0.8599 - precision: 0.8176 - recall: 0.9329